In this notebook code is provided to convert the raw data into epochs (stimulus and response centered)

# Preprocessing pipeline

#### Steps:
- STEP 1: Load raw files
- STEP 2: Visualize probes
- STEP 3: Remove white matter channels (and bads)
- STEP 4: Filter data
- STEP 5: Remove noisy contacts based on PSD
- STEP 6: Average reference the data
- STEP 7: Notch filter
- STEP 8: Create epoch object (stim and response centered)
- STEP 9: Visualize and remove channels and epochs


The code allows to load data of each participant to visually inspect and preprocess it in a semi automatized way.

In [ ]:
# Imports
import numpy as np
import pandas as pd
import mne
import matplotlib.pyplot as plt
# from mpl_toolkits.axes_grid1 import make_axes_locatable

import preprocessing_functions as ppf

import matplotlib
matplotlib.use('Qt5Agg') # This is to get interactive plots (very useful if using mne in a notebook)

### Load files

In [ ]:
# Load data (EXAMPLE OF SUBJ 1)
# data_path_sub1 = '../../Data/Raw + epochs + events/Subject_001/Data/260516/'
data_path_sub1 = '/Users/raimonbullich/Documents/Research_SPECS/Projects/The Unseen/Data/Raw + epochs + events/Subject_001/Data/260516/'

raw = mne.io.read_raw_fif(data_path_sub1 + '260516-raw.fif', preload=True) # raw iEEG file
eve = mne.read_events(data_path_sub1 + '260516-eve.fif') # raw events file
raw.add_events(eve) # Add events to raw

# Load electrodes df
# electrodes_path = '../../Data/Probes/S1_MMM.tsv'
electrodes_path = '/Users/raimonbullich/Documents/Research_SPECS/Projects/The Unseen/Data/Probes/S1_MMM.tsv'
electrodes_df = pd.read_csv(electrodes_path, sep='\t')

# Load behaviour data
events_sub1 = pd.read_pickle(data_path_sub1 + '260516_03-behavior.pickle') # behaviour


In [ ]:
# Plot and visualize raw data.
raw.plot()
# raw.plot(highpass=55, lowpass=65.0) # It can be useful to plot specific freq bands in case the raw data looks very noisy.
raw.info

### Remove white matter channels (and bads)

In [ ]:
# SELECT GRAY MATTER CONTACTS
gm_contact_name = electrodes_df.loc[~electrodes_df['aseg_reslice'].isin(['White L', 'White R']), 'Channel'] # get contacts from gray matter
raw_gm = raw.copy().pick_channels(gm_contact_name.values)

In [ ]:
# Visualize raw data for only gray matter channels
raw_gm.plot()

# NOTE: Using the interactive plot, one can select channels that look noisy, flat, ... and these will be removed from your raw object.


In [ ]:
raw_gm.info # if you have selected bad channels they will appear here

In [ ]:
# you can create a new object using only the good channels
bad_ch = raw_gm.info['bads']
raw_gm_good = raw_gm.copy().drop_channels(bad_ch)
raw_gm_good.info


### Visualize probes

In [ ]:
# plot in a 3d template the locations of the channels
all_gm_channels_list = raw_gm.info['ch_names']

dict_brain_regions = ppf.group_channels_brain_regions(electrodes_df, all_gm_channels_list)
list_of_list_brain_regions = [value for value in dict_brain_regions.values()]

ppf.plot_channel_locations(electrodes_df, all_gm_channels_list, 'All GM channels')
ppf.plot_channel_locations(electrodes_df, list_of_list_brain_regions, 'Brain regions GM channels')

### Remove noisy channels based on PSD

In [ ]:
# Visualize power spectrum density for all GM channels
raw_gm.plot_psd()
plt.title('PSD from "clean" channels')

# raw_gm.plot_psd()
# plt.title('PSD from all channels')
# plt.show()

# NOTE: selecting one channel in this plot will also be marked as bad.

### Filter data

In [ ]:
# Low freq = 0.1, high freq 150
raw_gm.filter(l_freq=0.1, h_freq=150)
# raw_gm_good.filter(l_freq=0.1, h_freq=150)

# NOTE: depending on what you want to do, downsampling the data can be a good idea (after all the preprocessing)

### Average reference the data

In [ ]:
mne.set_log_level('WARNING') # to not have many 'debug' or warning 'messages'

In [ ]:

# list_all_contacts_stim = raw_gm_good.info['ch_names']
# list_all_contacts_stim = raw_gm.info['ch_names']
list_all_contacts_stim = raw_gm_good.info['ch_names']
channels_list = ppf.create_probe_channels_list_of_lists(list_all_contacts_stim) # this is a list of lists

print(channels_list)


In [ ]:
# Average reference the data. This is done probe-wise.
# averaged_raw = ppf.average_reference_data(raw_gm, channels_list)
averaged_raw = ppf.average_reference_data(raw_gm_good, channels_list)


In [ ]:
# Visualize data after average reference
averaged_raw.plot()


### Notch filter 

In [ ]:
# Visualize psd after referencing
averaged_raw.plot_psd()

In [ ]:
# notch filter the data at 50Hz and its harmonics
averaged_raw_notch = averaged_raw.copy().notch_filter(freqs=[50, 100, 150])

In [ ]:
# Visualize result of notch filtering the data
averaged_raw_notch.plot_psd()


### Create epoch object (stimulus and response centered)

In [ ]:
# Plot raw data with annotations 
annotations = mne.annotations_from_events(eve, sfreq=averaged_raw_notch.info['sfreq'], first_samp=averaged_raw_notch.first_samp)
averaged_raw_notch.set_annotations(annotations)
averaged_raw_notch.plot()
averaged_raw_notch.info # I marked S2, M'15, and Q'12 as bads

# 2 is trial start
# 0 is flickering cross appears
# 1 is stimulus onset

In [ ]:
# Remove channels that still look very noisy
bad_ch = averaged_raw_notch.info['bads']
averaged_raw_notch_good = averaged_raw_notch.copy().drop_channels(bad_ch)

In [ ]:
# Create epoch object
raw_epochs_stimulus = mne.Epochs(averaged_raw_notch_good, eve, event_id=1, tmin=-1.5, tmax=2, baseline=None, reject=None, preload=True) # event id = 1 is when target is shown
# raw_epochs_stimulus = mne.Epochs(averaged_raw_notch, eve, event_id=1, tmin=-1.5, tmax=2, baseline=None, reject=None, preload=True) # event id = 1 is when target is shown
# Comment on baseline correction: once epochs are made, if needed, perform baseline correction by subtracting the mean amplitude of a pre-stimulus or pre-task interval from each epoch to reduce baseline drift effects.

# check that number of epochs and number of (behavior) trials match
print('len(events) == len(epochs) is:', len(events_sub1), len(raw_epochs_stimulus))

In [ ]:
raw_epochs_stimulus.plot(events=True)
raw_epochs_stimulus.info


In [ ]:
# Run this in case any epoch has been removed through epochs plot

# Find index of epochs that have been removed
# The issue is that the original indices of the epochs are not matching the events dataframe (cause many epochs have been removed due to missing target on cue).
# For this, I look at the original indices removed, and find the index where they belong in the selected epochs in this index array.

idx_removed = [ix for ix, log in enumerate(raw_epochs_stimulus.drop_log) if log == ('USER',)] # this provides me the idx of removed epochs
idx_saved = raw_epochs_stimulus.selection # here I have the idx of epochs I have saved
# print idx_saved to see the issue with original-current index and the mismatch between these and teh event df

all_idx = np.concatenate((idx_removed, idx_saved)) # concatenate both idx arrays
all_idx_sorted = np.sort(all_idx) # sort concatenated array
removed_epoch_idx = np.searchsorted(all_idx_sorted, idx_removed) # find indices of the values of idx_removed in the combined array all_idx_sorted

print(removed_epoch_idx)

### Remove epochs/channels based variance and on median absolute deviation

In [ ]:
raw_epochs_stimulus.plot()
# if you select one epoch after closing it gets removed (i removed the first one)

In [ ]:
####### CLEAN NOISY EPOCHS stimulus centered
ch_idx_stim, epoch_idx_stim = ppf.plot_variances_epochs_channels(raw_epochs_stimulus, 5) 

In [ ]:
# By selecting channels in this plot you can remove them. After, re run the previous cell.
# raw_epochs_stimulus.info.ch_names[72] # C'1 looks noisy, also S probe looks generally noisy
raw_epochs_stimulus.plot()

In [ ]:
# Plot epochs for a specific channel (choose how many, starting from where)
# ppf.plot_epochs_of_one_channel(raw_epochs_stimulus, ch_idx_stim, num_epochs=30, test_ch_idx=2)

In [ ]:
# Plot channels for one epoch
ppf.plot_channels_of_one_epoch(raw_epochs_stimulus, epoch_idx_stim[2], num_channels=20, test_epoch_idx=8)


In [ ]:
# Remove epochs or channels according to previous plots (and epoch drops from epoch.plot()). DOUBLE CHECK MATCH EPOCH IDX WITH TRIAL IDX FROM DF

len(events_sub1) == len(raw_epochs_stimulus)

events_clean = events_sub1.drop(removed_epoch_idx) # remove entries from removed epochs
len(events_clean) == len(raw_epochs_stimulus) # if this is true, seems like epochs and events match

In [ ]:
# SAVE THE EPOCHS AND NEW DATAFRAME
path_name_s1 = 'your/path'
events_clean.to_pickle(path_name_s1 + 'behaviour_s001.pickle')
raw_epochs_stimulus.save(path_name_s1 + 'raw_epochs_stim_s001.fif')

In [ ]:
# PLOT PSDs epochs
raw_epochs_stimulus.plot_psd()
